# Topics


In [1]:
import numpy as np
import pandas as pd
import math
import re
import sys
from shapely.geometry import Polygon
from matplotlib import pyplot as plt
from collections import Counter, OrderedDict, namedtuple, defaultdict, ChainMap
from queue import Queue
from copy import deepcopy
import networkx as nx
from functools import cmp_to_key, reduce
from itertools import product, permutations, combinations, combinations_with_replacement
from itertools import repeat
from functools import cache
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import maximum_flow
import json
import time
from tqdm import tqdm

In [2]:
sys.setrecursionlimit(1500)

In [3]:
with open("07-input", "r") as file:
    lines = file.readlines()
data_raw = "".join(lines)
# data_raw = [line.replace("\n", "") for line in lines]
# data_raw = "\n".join(data_raw)
data_raw

'25056746772: 4 47 136 21 79 49\n3790126111: 371 8 10 125 325 788\n37542888: 3 977 944 5\n785818128: 1 8 5 9 4 73 5 7 860 7\n19035366: 37 41 1 42 555 9 366\n1387: 33 502 69 775 5\n59640870978: 1 68 355 870 9 78\n10960521957: 7 369 8 6 4 1 5 23 53 5 9\n176844307608: 5 8 8 806 7 875 5 9 63 1\n3379786560: 3 2 349 1 6 4 7 5 4 16 9\n461716: 17 456 18 939 667\n7619: 2 5 9 671 6\n256034: 7 95 251 2 1 8\n106298: 21 156 11 5 6\n12374947048: 4 7 9 64 9 1 2 2 129 3 8 1\n16488497354846: 50 117 47 615 7 121 7\n15786760: 2 62 7 4 6 760\n25180862688: 874 335 5 8 7 1 36\n13552740: 1 1 51 9 535 595 1 6 4 9\n49121708165: 338 3 2 3 7 374 4 4 24 7\n138095646: 2 9 170 48 76\n564899635: 56 4 89 695 4 6 258 5\n69595110: 748 304 43 2 153\n55219024127: 492 1 28 745 5 6 8 2 5 5\n46695428: 9 2 5 8 9 373 4 49 1 7 8 3\n38256713: 1 17 565 7 569\n48706758: 994 49 7 6 1\n4201474: 8 194 2 10 9 9 3 5 3 9 2 2\n858412920: 8 8 8 372 8 728 3 1 8 6 9\n25236648217: 2 3 53 992 8 9 4 2 4 2 1 7\n527367299: 26 36 7 578 787 2 2\n

In [9]:
test_data_raw = r"""190: 10 19
3267: 81 40 27
83: 17 5
156: 15 6
7290: 6 8 6 15
161011: 16 10 13
192: 17 8 14
21037: 9 7 18 13
292: 11 6 16 20"""



def preprocess_data (data):
    # dtype='U10'
    
    def prep_row (row):
        left = int(row.split(":")[0])
        right = [int(number) for number in row.split(":")[1].split(" ") if number != ""]
        return left, right 
    
    rows =  [prep_row(row)  for row in data.split("\n")]
    return dict(rows)


test_data = preprocess_data(test_data_raw)
# test_data2 = preprocess_data(test_data_raw2)
display(test_data)

{190: [10, 19],
 3267: [81, 40, 27],
 83: [17, 5],
 156: [15, 6],
 7290: [6, 8, 6, 15],
 161011: [16, 10, 13],
 192: [17, 8, 14],
 21037: [9, 7, 18, 13],
 292: [11, 6, 16, 20]}

In [10]:
data = preprocess_data(data_raw)
data

{25056746772: [4, 47, 136, 21, 79, 49],
 3790126111: [371, 8, 10, 125, 325, 788],
 37542888: [3, 977, 944, 5],
 785818128: [1, 8, 5, 9, 4, 73, 5, 7, 860, 7],
 19035366: [37, 41, 1, 42, 555, 9, 366],
 1387: [33, 502, 69, 775, 5],
 59640870978: [1, 68, 355, 870, 9, 78],
 10960521957: [7, 369, 8, 6, 4, 1, 5, 23, 53, 5, 9],
 176844307608: [5, 8, 8, 806, 7, 875, 5, 9, 63, 1],
 3379786560: [3, 2, 349, 1, 6, 4, 7, 5, 4, 16, 9],
 461716: [17, 456, 18, 939, 667],
 7619: [2, 5, 9, 671, 6],
 256034: [7, 95, 251, 2, 1, 8],
 106298: [21, 156, 11, 5, 6],
 12374947048: [4, 7, 9, 64, 9, 1, 2, 2, 129, 3, 8, 1],
 16488497354846: [50, 117, 47, 615, 7, 121, 7],
 15786760: [2, 62, 7, 4, 6, 760],
 25180862688: [874, 335, 5, 8, 7, 1, 36],
 13552740: [1, 1, 51, 9, 535, 595, 1, 6, 4, 9],
 49121708165: [338, 3, 2, 3, 7, 374, 4, 4, 24, 7],
 138095646: [2, 9, 170, 48, 76],
 564899635: [56, 4, 89, 695, 4, 6, 258, 5],
 69595110: [748, 304, 43, 2, 153],
 55219024127: [492, 1, 28, 745, 5, 6, 8, 2, 5, 5],
 46695428: [

In [ ]:
def solution (data, verbose=False):
    total = 0
    
    for test, eq in data.items():
        # print(test, eq)
        valid_eq = False
        for ops in product("+*", repeat=len(eq)-1):
            term_list = [f"{num}){op}"  for num, op in zip(eq[:-1], ops)]

            term = "(" * (len(eq)-1) + "".join(term_list) + str(eq[-1])
            # print(term)
            # print(eval(term))
            if eval(term) == test:
                valid_eq = True
                break
        
        if valid_eq:
            total += test
    

    return total



sol = solution(test_data, verbose=True)

print(sol)


3749


In [41]:
sol = solution(data)
print(sol)

975671981569


# Part 2

In [ ]:
def solution2 (data, verbose=False):

    total = 0
    
    for test, eq in tqdm(data.items()):
           
        results = [eq[0]]
        for level in range(1, len(eq)):
            num = eq[level]
            results = [result*num   for result in results] + [result+num   for result in results] + [int(str(result)+str(num))   for result in results]
        
        if test in results:
            total += test
        

    return total



sol = solution2(test_data, verbose=True)
print(sol)
# print(dists)

# display(sum(sol))

    

100%|██████████| 9/9 [00:00<?, ?it/s]

11387


In [51]:
sol = solution2(data)
print(sol)

100%|██████████| 850/850 [00:05<00:00, 145.01it/s]

223472064194845
